In [35]:
from scipy.sparse import coo_matrix
a=coo_matrix(np.ones(shape=(10,10)),shape=(10,10))
b=coo_matrix(0,shape=(10,10))

ValueError: inconsistent shapes: (10, 10) != (1, 1)

In [5]:
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    '''callback to print loss after each epoch'''
    def __init__(self):
        self.epoch=0
        self.last_loss=0
    def on_epoch_end(self,model):
        loss=model.get_latest_training_loss()
        now_loss=loss-self.last_loss
        self.last_loss=loss
        test_word=list(model.wv.vocab.keys())[0]
        print('Loss after epoch {}: {}'.format(self.epoch,now_loss))
        print('for word {}: the array now is: {}'.format(test_word, model.wv.vectors[0][:10]))              
        print('for word {}: the top 3 sim user is {}'.format(test_word,model.wv.most_similar(test_word,topn=3)))                                                
        self.epoch+=1

In [7]:
from gensim.models import Word2Vec
model=Word2Vec.load('./item2vec-0.4a')
iv_df=pd.DataFrame(model.wv.vectors,index=model.wv.vocab.keys(),columns=['iv'+str(i) for i in range(0,128)]).reset_index()

In [11]:
iv_df['i2vec']=iv_df.iloc[:,1:].values.tolist()

In [17]:
iv_df.rename(columns={'index':'item_id'},inplace=True)

In [19]:
iv_df=iv_df[['item_id','i2vec']]

In [29]:
iv_df.shape
uv_df.shape

(98769, 2)

(31525, 2)

In [21]:
from gensim.models import Word2Vec
model=Word2Vec.load('./user2vec-0.4a')
uv_df=pd.DataFrame(model.wv.vectors,index=model.wv.vocab.keys(),columns=['iv'+str(i) for i in range(0,128)]).reset_index()

In [24]:
uv_df.rename(columns={'index':'user_id'},inplace=True)
uv_df['u2vec']=uv_df.iloc[:,1:].values.tolist()
uv_df=uv_df[['user_id','u2vec']]

In [28]:
uv_df

,user_id,u2vec
0,1489,"[-0.028489112854003906, -5.017876625061035, 3...."
1,10088,"[-4.466788291931152, -12.94594669342041, 7.529..."
2,13420,"[15.137311935424805, 5.448592185974121, -14.88..."
3,19312,"[-1.0070946216583252, -18.049047470092773, 6.8..."
4,31286,"[2.9684486389160156, -0.52272629737854, -8.469..."
...,...,...
31520,1518,"[-1.346155047416687, -0.9486507773399353, -0.0..."
31521,33187,"[0.3565931022167206, -0.7588027119636536, 0.29..."
31522,26614,"[-0.9818758964538574, -1.3186801671981812, 0.8..."
31523,31461,"[-0.7148497104644775, -1.5795958042144775, 1.0..."


In [26]:
user_path='../user_data/'

In [27]:
uv_df.to_csv(user_path+'user_embedding.csv',index=False)
iv_df.to_csv(user_path+'item_embedding.csv',index=False)

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from loguru import logger
import argparse


class PickleWrapper(object):

    @classmethod
    def loadFromFile(cls, file, mode='rb'):
        with open(file, mode) as f:
            return pickle.load(f)

    @classmethod
    def dump2File(cls, o, file, mode='wb'):
        '''
        把目标对象序列化到文件
        :param o: 目标对象
        :param file: 文件
        :param mode:
        :return:
        '''
        with open(file, mode) as f:
            pickle.dump(o, f)



parser = argparse.ArgumentParser(description='t')
parser.add_argument('--now_phase', type=int, default=6, help='')
parser.add_argument('--window', type=int, default=10, help='cocur_matr的时间窗')
parser.add_argument('--time_decay', type=float, default=7/8, help='时间衰减')
parser.add_argument('--mode', type=str, default='train', help='train test')
parser.add_argument('--topk', type=int, default=500, help='每种召回策略召回的样本数')
parser.add_argument('--DATA_DIR', type=str, default='./', help='data dir')

args = parser.parse_args(args=[])
trace = logger.add(os.path.join(args.DATA_DIR, 'data_gen/runtime.log'))

# Cell
def load_click_data_per_phase(now_phase, base_dir):
    """
    """


    all_click_df = []
    for c in range(now_phase + 1):
        logger.info(f'phase: {c}')
        cols_str = 'user_id item_id time'.split()
        click_train1 = pd.read_csv( './underexpose_train_click-{}.csv'.format(c), header=None,  names=['user_id', 'item_id', 'time'], converters={c: str for c in cols_str})
        click_test1 = pd.read_csv( './underexpose_test_click-{}.csv'.format(c, c), header=None,  names=['user_id', 'item_id', 'time'], converters={c: str for c in cols_str})
        test_qtime1 = pd.read_csv( './underexpose_test_qtime-{}.csv'.format(c, c), header=None,  names=['user_id','time'], converters={c: str for c in cols_str})
        click_test1_val = click_test1.sort_values(['user_id', 'time']).drop_duplicates(subset=['user_id'],keep='last')

        click_test1 = click_test1[~click_test1.index.isin(click_test1_val.index)]
        all_click = click_train1.append(click_test1).drop_duplicates().sort_values('time')

        all_click_df.append((all_click, click_test1_val, test_qtime1))
        logger.info(f'all_click: {all_click.shape}, click_test1_val: {click_test1_val.shape}, test_qtime1: {test_qtime1.shape}')
    return all_click_df
all_click_df = load_click_data_per_phase(args.now_phase, args.DATA_DIR)
all_train=pd.concat([all_click_df[i][0] for i in range(0,7)]).drop_duplicates().reset_index(drop=True)


# item2vec=train6.groupby('user_id')['item_id'].agg(list)
# item2vec
user2vec=all_train.groupby('item_id')['user_id'].agg(list)
user2vec

sentences=list(user2vec.values)

import multiprocessing
import gensim
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    '''callback to print loss after each epoch'''
    def __init__(self):
        self.epoch=0
        self.last_loss=0
    def on_epoch_end(self,model):
        loss=model.get_latest_training_loss()
        now_loss=loss-self.last_loss
        self.last_loss=loss
        test_word=list(model.wv.vocab.keys())[0]
        print('Loss after epoch {}: {}'.format(self.epoch,now_loss))
        print('for word {}: the array now is: {}'.format(test_word, model.wv.vectors[0][:10]))              
        print('for word {}: the top 3 sim user is {}'.format(test_word,model.wv.most_similar(test_word,topn=3)))                                                
        self.epoch+=1

model = gensim.models.Word2Vec(sentences,
                    size=128,
                    alpha=0.4,
                    window=999999,
                    min_count=1,
                    workers=multiprocessing.cpu_count(),
                    compute_loss=True,
                    iter=27,
                    hs=0,
                    sg=1,
                    seed=42,callbacks=[callback()])
model.save('./user2vec-0.4')





_StoreAction(option_strings=['--now_phase'], dest='now_phase', nargs=None, const=None, default=6, type=<class 'int'>, choices=None, help='', metavar=None)

_StoreAction(option_strings=['--window'], dest='window', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help='cocur_matr的时间窗', metavar=None)

_StoreAction(option_strings=['--time_decay'], dest='time_decay', nargs=None, const=None, default=0.875, type=<class 'float'>, choices=None, help='时间衰减', metavar=None)

_StoreAction(option_strings=['--mode'], dest='mode', nargs=None, const=None, default='train', type=<class 'str'>, choices=None, help='train test', metavar=None)

_StoreAction(option_strings=['--topk'], dest='topk', nargs=None, const=None, default=500, type=<class 'int'>, choices=None, help='每种召回策略召回的样本数', metavar=None)

_StoreAction(option_strings=['--DATA_DIR'], dest='DATA_DIR', nargs=None, const=None, default='./', type=<class 'str'>, choices=None, help='data dir', metavar=None)

2020-06-04 13:22:27.013 | INFO     | __main__:load_click_data_per_phase:52 - phase: 0
2020-06-04 13:22:27.692 | INFO     | __main__:load_click_data_per_phase:63 - all_click: (261337, 3), click_test1_val: (1663, 3), test_qtime1: (1663, 2)
2020-06-04 13:22:27.694 | INFO     | __main__:load_click_data_per_phase:52 - phase: 1
2020-06-04 13:22:28.359 | INFO     | __main__:load_click_data_per_phase:63 - all_click: (264871, 3), click_test1_val: (1726, 3), test_qtime1: (1726, 2)
2020-06-04 13:22:28.360 | INFO     | __main__:load_click_data_per_phase:52 - phase: 2
2020-06-04 13:22:29.041 | INFO     | __main__:load_click_data_per_phase:63 - all_click: (264624, 3), click_test1_val: (1690, 3), test_qtime1: (1690, 2)
2020-06-04 13:22:29.043 | INFO     | __main__:load_click_data_per_phase:52 - phase: 3
2020-06-04 13:22:29.795 | INFO     | __main__:load_click_data_per_phase:63 - all_click: (286609, 3), click_test1_val: (1675, 3), test_qtime1: (1675, 2)
2020-06-04 13:22:29.797 | INFO     | __main__:lo

item_id
1                              [1489, 10088, 13420, 19312]
10                                    [31286, 8401, 16881]
100          [32252, 6591, 19538, 9914, 790, 13780, 10602]
1000     [14546, 9531, 32114, 6934, 20558, 30360, 33492...
10000                          [26044, 24047, 5063, 22077]
                               ...                        
99994    [9287, 6305, 28964, 17648, 22420, 15431, 32017...
99996                          [7266, 25455, 25016, 13092]
99997                    [20924, 6843, 15103, 3138, 28030]
99998    [24217, 9796, 20948, 25551, 16599, 33363, 2946...
99999    [31021, 21028, 11225, 14902, 16578, 11050, 225...
Name: user_id, Length: 98769, dtype: object

Loss after epoch 0: 25268.736328125
for word 1489: the array now is: [-1.659646   -4.375808    1.5798957  -3.4722328   1.9806672   4.901176
  5.6547112   0.23472086 -3.1559265  -1.7262268 ]
for word 1489: the top 3 sim user is [('18806', 0.8132525682449341), ('5374', 0.8051040172576904), ('29258', 0.8050020933151245)]
Loss after epoch 1: 3065.96875
for word 1489: the array now is: [-1.0222683 -5.3737926  1.9490113 -9.064284   4.053353   4.707161
  6.033243   1.5210111 -3.2908812 -2.001929 ]
for word 1489: the top 3 sim user is [('18806', 0.8132525682449341), ('5374', 0.8051040172576904), ('29258', 0.8050020933151245)]
Loss after epoch 2: 1286.427734375
for word 1489: the array now is: [-0.9802516 -5.398981   2.0609684 -9.088509   3.8956044  4.7317953
  5.955675   1.6005725 -3.3330247 -2.0296798]
for word 1489: the top 3 sim user is [('18806', 0.8132525682449341), ('5374', 0.8051040172576904), ('29258', 0.8050020933151245)]
Loss after epoch 3: 1062.26171875
for word 1489: the array now 

NameError: name 'test6' is not defined

In [7]:


user_train=all_click_df[6][0].sort_values(by=['user_id','time'],ascending=[True,False])
user_true=all_click_df[6][1] 
recall_frame = pd.DataFrame(columns=['user_id', 'item_id_pred', 'score', 'rank', 'item_id_true'],index=range(0,500*len(user_true.user_id.unique())))
search_df=user_train.drop_duplicates(['user_id'],keep='first')
user_list=search_df.user_id.unique()






row_num=0
for user in tqdm(user_true.user_id.unique()):
    true_item = user_true.loc[user_true.user_id == user, 'item_id'].values[0]
    i=0
    for user_sim,score in model.wv.most_similar(user,topn=500):
        if user_sim in user_list:
            recall_frame.iloc[row_num]['user_id'] = user
            recall_frame.iloc[row_num]['item_id_pred'] = search_df.loc[search_df.user_id == user_sim,'item_id'].values[0]
            recall_frame.iloc[row_num]['score'] = score
            recall_frame.iloc[row_num]['rank'] = i+1
            recall_frame.iloc[row_num]['item_id_true'] = true_item
            row_num+=1
sum(recall_frame.item_id_pred == recall_frame.item_id_true)/1821




100%|██████████| 1821/1821 [29:57<00:00,  1.01it/s]


0.030752333882482153

In [37]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from loguru import logger
import argparse
class PickleWrapper(object):

    @classmethod
    def loadFromFile(cls, file, mode='rb'):
        with open(file, mode) as f:
            return pickle.load(f)

    @classmethod
    def dump2File(cls, o, file, mode='wb'):
        '''
        把目标对象序列化到文件
        :param o: 目标对象
        :param file: 文件
        :param mode:
        :return:
        '''
        with open(file, mode) as f:
            pickle.dump(o, f)

# Cell


def get_item_data():
    train_item_df = pd.read_csv( './underexpose_item_feat.csv', sep=r',\s+|,\[|\],\[',names=['item_id']+list(range(256)),converters={'item_id':str})
    train_item_df.iloc[:, -1] = train_item_df.iloc[:, -1].str.replace(']', '').map(float)
    return train_item_df
# Cell
def val1(df, topk, click_val):
    """
    """
    sr = df.groupby('user_id')['item_id_pred'].agg(list)
    r_list = []
    pred_num = click_val.shape[0]
    for line in click_val.itertuples():
        try:
            df.loc[df.user_id==line.user_id, 'item_id_true'] = line.item_id
            t = sr[line.user_id]
            i = t.index(line.item_id)
        except:
            i = 99999

        r_list.append(i)
    r_series = pd.Series(r_list)
    for i in range(50, topk+1, 50):
        logger.info(f'recall{i}:{r_series[r_series<i].shape[0]/pred_num}')

#     r_series[r_series<9999].plot(kind='box', title='pp', rot=45)
    return df



# Cell
def load_match_items():
    r_itemcf = PickleWrapper.loadFromFile('./r_list_itemcf_0527.pkl'))

    r_binn = PickleWrapper.loadFromFile( './r_list_binn_0527.pkl'))

#     r_itemcf_phase4 = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_0527_phase4.pkl'))

#     r_binn_phase4 = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_0527_phase4.pkl'))


    r_itemcf_yl = PickleWrapper.loadFromFile('./r_list_itemcf_yulao_0527.pkl'))

    # r_itemcf = r_itemcf + r_itemcf_phase4
    # r_binn = r_binn + r_binn_phase4
    r_itemcf_yl = [r_itemcf_yl[0], r_itemcf_yl[1],r_itemcf_yl[2],r_itemcf_yl[3],r_itemcf_yl[5],r_itemcf_yl[6],r_itemcf_yl[4]]
    return r_itemcf, r_binn, r_itemcf_yl
#     return r_itemcf_phase4, r_binn_phase4, [r_itemcf_yl[4]]


# Cell

def data_preporcess(recall_list, match_num, phase, mode='train'):

    itemcf, r_binn, r_itemcf_yl = recall_list
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    for i in range(len(phase)):
        r_itemcf[i]['phase'] = phase[i]
        r_binn[i]['phase'] = phase[i]
        r_itemcf_yl[i]['phase'] = phase[i]
        df1 = df1.append(r_itemcf[i])
        df2 = df2.append(r_binn[i])
        df3 = df3.append(r_itemcf_yl[i])
        
    df1_ = df1[df1['rank']<(match_num+1)]
    df2_ = df2[df2['rank']<(match_num+1)]
    df3_ = df3[df3['rank']<(match_num+1)]

    print('merge Multi-channel recall...')
    cols = 'user_id item_id_pred phase'.split()

    if mode == 'train':
        cols = 'user_id item_id_pred item_id_true phase'.split()
    df = pd.merge(pd.merge(df1_, df2_, on=cols, how='outer'),df3_, on=cols, how='outer')

    df = df.fillna(0)
    if mode == 'train':
        df['label'] = (df['item_id_pred'] == df['item_id_true']).map(int)


    dft = pd.DataFrame()
    for p in tqdm(phase):
        # 获取test用户的近n次点击
        temp_ = all_click_df[p][0][all_click_df[p][0].user_id.isin(all_click_df[p][1].user_id)].groupby('user_id')['item_id'].agg(list)
        temp_ = pd.DataFrame(temp_)
        for i in range(0, 5):
            temp_[f'last_{i+1}'] = temp_.item_id.str.get(-(i+1))
        dft = dft.append(temp_.reset_index())
        # 计算每个phase的item_id cnt
        t_ = all_click_df[p][0].groupby('item_id')['user_id'].count()
        df.loc[df.phase==p, 'item_cnt'] = df.item_id_pred.map(lambda x: t_[x] if x in t_ else 0)

    df = pd.merge(df, dft.drop(columns='item_id'))

    df['item_id_pred_text_vec'] = df.item_id_pred.map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

    df['item_id_pred_img_vec'] = df.item_id_pred.map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))

    for c in tqdm('last_1 last_2 last_3'.split()):
        df[f'{c}_text_vec'] = df.item_id_pred.map(lambda x: item_feat['text_vec'][x] if x in item_feat['text_vec'] else np.zeros(128))

        df[f'{c}_img_vec'] = df.item_id_pred.map(lambda x: item_feat['img_vec'][x] if x in item_feat['img_vec'] else np.arange(128))


    # 太大了，不建议存储
    # df.to_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}.pkl'))

    # df = pd.read_pickle(os.path.join(args.DATA_DIR, f'data_gen/dft{match_num}.pkl'))
    return df

# Cell
def fe(df):
    for i in tqdm('rank score'.split()):
        df[f'{i}1_sub_{i}2'] = df[f'{i}_x']-df[f'{i}_y']
        df[f'{i}1_add_{i}2'] = df[f'{i}_x']+df[f'{i}_y']
        df[f'{i}1_mul_{i}2'] = df[f'{i}_x']*df[f'{i}_y']

        df[f'{i}1_sub_{i}3'] = df[f'{i}_x']-df[f'{i}']
        df[f'{i}1_add_{i}3'] = df[f'{i}_x']+df[f'{i}']
        df[f'{i}1_mul_{i}3'] = df[f'{i}_x']*df[f'{i}']

        df[f'{i}2_sub_{i}3'] = df[f'{i}_y']-df[f'{i}']
        df[f'{i}2_add_{i}3'] = df[f'{i}_y']+df[f'{i}']
        df[f'{i}2_mul_{i}3'] = df[f'{i}_y']*df[f'{i}']

    df['sim1_text'] = (df['item_id_pred_text_vec'] * df['last_1_text_vec']).map(sum)

    df['sim1_img'] = (df['item_id_pred_img_vec'] * df['last_1_img_vec']).map(sum)

    df['sim2_text'] = (df['item_id_pred_text_vec'] * df['last_2_text_vec']).map(sum)
    df['sim2_img'] = (df['item_id_pred_img_vec'] * df['last_2_img_vec']).map(sum)

    df['sim3_text'] = (df['item_id_pred_text_vec'] * df['last_3_text_vec']).map(sum)
    df['sim3_img'] = (df['item_id_pred_img_vec'] * df['last_3_img_vec']).map(sum)

#     df['sim4_text'] = (df['item_id_pred_text_vec'] * df['last_4_text_vec']).map(sum)
#     df['sim4_img'] = (df['item_id_pred_img_vec'] * df['last_4_img_vec']).map(sum)

#     df['sim5_text'] = (df['item_id_pred_text_vec'] * df['last_5_text_vec']).map(sum)
#     df['sim5_img'] = (df['item_id_pred_img_vec'] * df['last_5_img_vec']).map(sum)



    df['sim1_text_img'] = df['sim1_text'] *  df['sim1_img']
    df['sim2_text_img'] = df['sim2_text'] *  df['sim2_img']
    df['sim3_text_img'] = df['sim3_text'] *  df['sim3_img']
#     df['sim4_text_img'] = df['sim4_text'] *  df['sim4_img']
#     df['sim5_text_img'] = df['sim5_text'] *  df['sim5_img']

    df['sim12_text'] = df['sim1_text'] + df['sim2_text']
    df['sim123_text'] = df['sim1_text'] + df['sim2_text'] + df['sim3_text']
#     df['sim1234_text'] = df['sim1_text'] + df['sim2_text'] + df['sim3_text'] + df['sim4_text']
#     df['sim12345_text'] = df['sim1_text'] + df['sim2_text'] + df['sim3_text'] + df['sim4_text'] + df['sim5_text']

    df['sim12_img'] = df['sim1_img'] + df['sim2_img']
    df['sim123_img'] = df['sim1_img'] + df['sim2_img'] + df['sim3_img']
#     df['sim1234_img'] = df['sim1_img'] + df['sim2_img'] + df['sim3_img'] + df['sim4_img']
#     df['sim12345_img'] = df['sim1_img'] + df['sim2_img'] + df['sim3_img'] + df['sim4_img'] + df['sim5_img']

    df['sim12_text_img'] = df['sim1_text_img'] + df['sim2_text_img']
    df['sim123_text_img'] = df['sim1_text_img'] + df['sim2_text_img'] + df['sim3_text_img']
#     df['sim1234_text_img'] = df['sim1_text_img'] + df['sim2_text_img'] + df['sim3_text_img'] + df['sim4_text_img']
#     df['sim12345_text_img'] = df['sim1_text_img'] + df['sim2_text_img'] + df['sim3_text_img'] + df['sim4_text_img'] + df['sim5_text_img']


    return df

# Cell
def get_model_input(df, mode='train'):
    from catboost import Pool, CatBoostClassifier

    cols = ['score_x', 'rank_x','score_y', 'rank_y', 'score', 'rank', 'item_cnt','sim1_text', 'sim1_img', 'sim2_text','sim2_img', 'sim3_text', 'sim3_img', 'sim4_text', 'sim4_img','sim5_text', 'sim5_img', 'rank1_sub_rank2', 'rank1_add_rank2','rank1_mul_rank2', 'rank1_sub_rank3', 'rank1_add_rank3','rank1_mul_rank3', 'rank2_sub_rank3', 'rank2_add_rank3','rank2_mul_rank3', 'score1_sub_score2', 'score1_add_score2','score1_mul_score2', 'score1_sub_score3', 'score1_add_score3','score1_mul_score3', 'score2_sub_score3', 'score2_add_score3','score2_mul_score3', 'sim1_text_img', 'sim2_text_img', 'sim3_text_img','sim4_text_img', 'sim5_text_img', 'sim12_text', 'sim123_text','sim1234_text', 'sim12345_text', 'sim12_img', 'sim123_img','sim1234_img', 'sim12345_img', 'sim12_text_img', 'sim123_text_img','sim1234_text_img', 'sim12345_text_img']
    
    df['fscore'] = 0
    df['frank'] = 999
    cat_cols = []
    if mode=='train':
        X_train, X_test, y_train, y_test = df.loc[df['phase']<6, cols], df.loc[df['phase']==6, cols], df.loc[df['phase']<6, 'label'],df.loc[df['phase']==6, 'label']


        train_data = Pool(data=X_train,label=y_train,cat_features=cat_cols)
        val_data = Pool(data=X_test, label=y_test, cat_features=cat_cols)
        return train_data, val_data, cat_cols
    X_test = df[cols]
    test_data = Pool(data=X_test, cat_features=cat_cols)
    return test_data, cat_cols

# Cell
def get_model():
    from catboost import Pool, CatBoostClassifier
    params = {'iterations': 360,'learning_rate': 0.1,'random_seed': 144,'custom_metric': 'F1','loss_function': 'Logloss','class_weights': [1, 20],}
    print(params)
    model = CatBoostClassifier(**params)
    return model

# Cell
if __name__ == '__main__':

    parser = argparse.ArgumentParser(description='t')
    parser.add_argument('--now_phase', type=int, default=6, help='')
    parser.add_argument('--window', type=int, default=10, help='cocur_matr的时间窗')
    parser.add_argument('--time_decay', type=float, default=7/8, help='时间衰减')
    parser.add_argument('--mode', type=str, default='train', help='train test')
    parser.add_argument('--topk', type=int, default=500, help='每种召回策略召回的样本数')
    parser.add_argument('--DATA_DIR', type=str, default='../../data/contest/kdd_debiasing', help='data dir')

    args = parser.parse_args(args=[])
    trace = logger.add(os.path.join(args.DATA_DIR, 'data_gen/runtime.log'))
    # Cell
    all_click_df = load_click_data_per_phase(args.now_phase, args.DATA_DIR)

    # Cell
    item_feat = get_item_data()

    # Cell
    item_feat['text_vec'] = item_feat.iloc[:, 1:129].values.tolist()
    item_feat['img_vec'] = item_feat.iloc[:, 129:257].values.tolist()

    item_feat['text_vec'] = item_feat['text_vec'].map(np.array)

    item_feat['img_vec'] = item_feat['img_vec'].map(np.array)

    item_feat.set_index('item_id', inplace=True)

    # Cell
    phase = [0, 1, 2, 3, 5, 6]

    # phase = [4]
    # match_num = 200
    match_num = 500
   

_StoreAction(option_strings=['--now_phase'], dest='now_phase', nargs=None, const=None, default=6, type=<class 'int'>, choices=None, help='', metavar=None)

_StoreAction(option_strings=['--window'], dest='window', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help='cocur_matr的时间窗', metavar=None)

_StoreAction(option_strings=['--time_decay'], dest='time_decay', nargs=None, const=None, default=0.875, type=<class 'float'>, choices=None, help='时间衰减', metavar=None)

_StoreAction(option_strings=['--mode'], dest='mode', nargs=None, const=None, default='train', type=<class 'str'>, choices=None, help='train test', metavar=None)

_StoreAction(option_strings=['--topk'], dest='topk', nargs=None, const=None, default=500, type=<class 'int'>, choices=None, help='每种召回策略召回的样本数', metavar=None)

_StoreAction(option_strings=['--DATA_DIR'], dest='DATA_DIR', nargs=None, const=None, default='../../data/contest/kdd_debiasing', type=<class 'str'>, choices=None, help='data dir', metavar=None)

2020-06-04 14:18:14.956 | INFO     | __main__:load_click_data_per_phase:52 - phase: 0
2020-06-04 14:18:16.293 | INFO     | __main__:load_click_data_per_phase:63 - all_click: (261337, 3), click_test1_val: (1663, 3), test_qtime1: (1663, 2)
2020-06-04 14:18:16.295 | INFO     | __main__:load_click_data_per_phase:52 - phase: 1
2020-06-04 14:18:17.595 | INFO     | __main__:load_click_data_per_phase:63 - all_click: (264871, 3), click_test1_val: (1726, 3), test_qtime1: (1726, 2)
2020-06-04 14:18:17.597 | INFO     | __main__:load_click_data_per_phase:52 - phase: 2
2020-06-04 14:18:18.399 | INFO     | __main__:load_click_data_per_phase:63 - all_click: (264624, 3), click_test1_val: (1690, 3), test_qtime1: (1690, 2)
2020-06-04 14:18:18.401 | INFO     | __main__:load_click_data_per_phase:52 - phase: 3
2020-06-04 14:18:19.153 | INFO     | __main__:load_click_data_per_phase:63 - all_click: (286609, 3), click_test1_val: (1675, 3), test_qtime1: (1675, 2)
2020-06-04 14:18:19.155 | INFO     | __main__:lo

FileNotFoundError: [Errno 2] No such file or directory: '../../data/contest/kdd_debiasing/data_gen/r_list_itemcf_0527.pkl'

In [ ]:
def load_match_items():
    r_itemcf = PickleWrapper.loadFromFile('./r_list_itemcf_0527.pkl')

    r_binn = PickleWrapper.loadFromFile( './r_list_binn_0527.pkl')

#     r_itemcf_phase4 = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_itemcf_0527_phase4.pkl'))

#     r_binn_phase4 = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/r_list_binn_0527_phase4.pkl'))


    r_itemcf_yl = PickleWrapper.loadFromFile('./r_list_itemcf_yulao_0527.pkl')

    # r_itemcf = r_itemcf + r_itemcf_phase4
    # r_binn = r_binn + r_binn_phase4
    r_itemcf_yl = [r_itemcf_yl[0], r_itemcf_yl[1],r_itemcf_yl[2],r_itemcf_yl[3],r_itemcf_yl[5],r_itemcf_yl[6],r_itemcf_yl[4]]
    return r_itemcf, r_binn, r_itemcf_yl
#     return r_itemcf_phase4, r_binn_phase4, [r_itemcf_yl[4]]


r_itemcf, r_binn, r_itemcf_yl = load_match_items()
df = data_preporcess([r_itemcf, r_binn, r_itemcf_yl], match_num, phase)
df = fe(df)
train_data, val_data, cat_cols = get_model_input(df)
model = get_model()
r = model.fit(train_data,verbose=False,plot=False)
df.loc[df['phase']==6, 'fscore'] = model.predict_proba(val_data)[:, 1]

df.loc[df['phase']==6, 'frank'] = df.loc[df['phase']==6].groupby('user_id')['fscore'].rank(method='first', ascending=False)
r = val1(df.loc[(df['phase']==6)&(df['frank']<101)], 100, all_click_df[6][1])

merge Multi-channel recall...


 83%|████████▎ | 5/6 [10:29<02:10, 130.42s/it]